# 中间件flask-restaction

flask-restful是比较老比较成熟的中间件了,而[`flask-restaction`](http://flask-restaction.readthedocs.io/zh/latest/quickstart.html)对其做了改进你并且更加优雅好用

它的优势有:

优势|说明
---|---
restaction 更灵活|restful 的方法只能是 http method，就是 get, post, put, delete 那几个，而 restaction 的方法除了 http method，还可以是任何以 http method 加下划线开头的方法。
输入输出校验|restaction 是声明式的，简单明确,Request Parsing 很繁琐，并且不能很好的重用代码。restaction 的输出校验和输入校验差不多，不同的是可以校验自定义的 python 对象。` https://github.com/guyskk/validater#proxydict-validate-custome-type`, 而 reslful 校验输出更加繁琐！
身份验证及权限控制|restaction 提供一个灵活的权限系统，身份验证基于 jwt(json web token)， 权限验证是通过json配置文件，而不是散布在代码中的装饰器(decorator)， 并且角色本身也是 resource，客户端可以通过 API 进行操作。
自动生成文档，res.js和权限管理页面|用 res.js 可以方便的调用 api，还可以直接上传文件。


> 使用`flask-restaction`改写user的增删改查服务

In [55]:
%%writefile codes/c5/app.py
# --*-- coding:utf-8 --*--
from __future__ import print_function, unicode_literals, division

from flask import Flask,jsonify,request
from flask_restaction import Resource, Api, Gen, Res
import json

app = Flask(__name__)
api = Api(app)
res = Res(api)

Userlist = []
def inc(cls):
    counter = {"result":0}
    def wrap(*args):
        counter["result"]+=1
        result = cls(counter["result"],*args)
        global Userlist
        Userlist.append(result)
        return result
    return wrap
@inc
class User(object):
    def __str__(self):
        return "<USER: id-{self._id}-{self.name}>".format(self=self)
    def __repr__(self):
        return self.__str__()
    def __init__(self,_id,name,password,type="local"):
        self._id=_id
        self.name = name
        self.password = password
        self.type=type
             
    def toDict(self):
        return {"id": self._id,
        "name": self.name,
        "password": self.password,
        "type":self.type}
@app.errorhandler(404)
def page_not_found(error):
    return jsonify({"message":"404 not found","code":404}), 404


class Users(Resource):
    paging = {
        "pagenum": "+int&default=1",
        "pagesize": "int(5,50)&default=10"
    }
    info = {
        "id": "+int&required",
        "name": "safestr&required",
        "password": "safestr&required",
        "type":"safestr&required"
    }
    schema_inputs = {
        "get": {"user_id": "+int&required"},
        "post": {
            "name": ("safestr&required", "your name"),
            "password":("safestr&required", "your password")
        },
        "put": {
            "user_id": "+int&required",
            "name": "safestr",
            "password": "safestr"
        },
        "delete": {"user_id": "+int&required"}
    }
    schema_outputs = {
        "get": info,
        "get_list": [info],
        "post": info,
        "put": info,
        "delete": {"message": "unicode"}
    }
    def get(self,user_id):
        try:
            user = filter(lambda x:True if x._id==user_id else False,Userlist)[0]
        except IndexError as e:
            abort(404)
        return user.toDict()
    
    def get_list(self):
        return map(lambda x:x.toDict(),Userlist)
        
    def post(self,name,password):
        print(name)
        user = User(name,password)
        return user.toDict()
    
     
    def put(self,user_id,name,password):
        user = filter(lambda x: True if x._id==user_id else False,Userlist)[0]
        if name:
            user.name=name
        if password:
            user.password=password
        
        return user.toDict()
    def delete(self,user_id):
        global Userlist
        Userlist = filter(lambda x: False if x._id==user_id else True,Userlist)
        return {"message": "OK"}
        
api.add_resource(Users)

gen = Gen(api)
gen.resjs('static/js/res.js')
gen.resdocs('static/resdocs.html', resjs='/static/js/res.js',
                bootstrap='/static/css/bootstrap.min.css')

if __name__ == '__main__':
    app.run(debug=True)


Overwriting codes/c5/app.py


In [56]:
import requests
import json

In [57]:
requests.post("http://127.0.0.1:5000/users",data = json.dumps({"name":"niuniu","password":"qwe"}),
              headers={'content-type': 'application/json'}).json()

{u'id': 1, u'name': u'niuniu', u'password': u'qwe', u'type': u'local'}

In [58]:
requests.post("http://127.0.0.1:5000/users",data = json.dumps({"name":"baba","password":"qwe"}),
              headers={'content-type': 'application/json'}).json()

{u'id': 2, u'name': u'baba', u'password': u'qwe', u'type': u'local'}

In [59]:
requests.post("http://127.0.0.1:5000/users",data = json.dumps({"name":"lala","password":"qwe"}),
              headers={'content-type': 'application/json'}).json()

{u'id': 3, u'name': u'lala', u'password': u'qwe', u'type': u'local'}

In [60]:
requests.post("http://127.0.0.1:5000/users",data = json.dumps({"name":"nubo","password":"qwe"}),
              headers={'content-type': 'application/json'}).json()

{u'id': 4, u'name': u'nubo', u'password': u'qwe', u'type': u'local'}

In [61]:
requests.get("http://127.0.0.1:5000/users/list").json()

[{u'id': 1, u'name': u'niuniu', u'password': u'qwe', u'type': u'local'},
 {u'id': 2, u'name': u'baba', u'password': u'qwe', u'type': u'local'},
 {u'id': 3, u'name': u'lala', u'password': u'qwe', u'type': u'local'},
 {u'id': 4, u'name': u'nubo', u'password': u'qwe', u'type': u'local'}]

In [62]:
requests.get("http://127.0.0.1:5000/users",params={'user_id':1}).json()

{u'id': 1, u'name': u'niuniu', u'password': u'qwe', u'type': u'local'}

In [63]:
requests.put("http://127.0.0.1:5000/users",
             data = json.dumps({'user_id':1,"name":"fadf"}), 
             headers={'content-type': 'application/json'}).json()

{u'id': 1, u'name': u'fadf', u'password': u'qwe', u'type': u'local'}

In [64]:
requests.get("http://127.0.0.1:5000/users/list").json()

[{u'id': 1, u'name': u'fadf', u'password': u'qwe', u'type': u'local'},
 {u'id': 2, u'name': u'baba', u'password': u'qwe', u'type': u'local'},
 {u'id': 3, u'name': u'lala', u'password': u'qwe', u'type': u'local'},
 {u'id': 4, u'name': u'nubo', u'password': u'qwe', u'type': u'local'}]

In [65]:
requests.delete("http://127.0.0.1:5000/users",params={'user_id':1}).json()

{u'message': u'OK'}

In [66]:
requests.get("http://127.0.0.1:5000/users/list").json()

[{u'id': 2, u'name': u'baba', u'password': u'qwe', u'type': u'local'},
 {u'id': 3, u'name': u'lala', u'password': u'qwe', u'type': u'local'},
 {u'id': 4, u'name': u'nubo', u'password': u'qwe', u'type': u'local'}]